In [1]:
import random
import numpy as np
import time
from scipy import sparse
import random
from importlib import reload
from recapi import recommender as rec

In [2]:
# Load the books dataset
import csv
incidence_file = '../data/csv/BX-Book-Ratings.csv'
user_item_rating_triples = list()
with open(incidence_file, encoding='latin1') as f:
    r = csv.reader(f, delimiter=';', skipinitialspace=True, quoting=csv.QUOTE_ALL)
    next(r)
    for row in r:
        if row[2] != '0' and row[2] < '5':
            continue
        user_item_rating_triples.append((row[0], row[1], 1))

In [ ]:
# movielens dataset
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')
user_item_rating_triples = [(u, i, 1) for u, i, r, t in data.raw_ratings if int(t) > 1]

In [ ]:
reload(rec)
%time rmdr = rec.build_recommender_from_triples(\
    user_item_rating_triples, min_item_freq=5, min_user_freq=3)

print('%d items left' % len(rmdr.items))
print('%d users left' % len(rmdr.users))

In [ ]:
reload(rec)
seed = random.randint(0,1000)
eval_str = 'mean relative rank %.4f, std %.3f'

seed = 15
alpha = 3
beta = 0.01

samples = 1000

%time ranks = rec.ranking_stat(rmdr, samples, seed, alpha=alpha, no_mat=False, no_bias=True)
print(eval_str % (np.mean(ranks), np.std(ranks)))

%time ranks = rec.ranking_stat(rmdr, samples, seed, alpha=alpha, beta=beta, no_mat=False, no_bias=False)
print(eval_str % (np.mean(ranks), np.std(ranks)))

%time ranks = rec.ranking_stat(rmdr, samples, seed, no_mat=True, no_bias=False)
print(eval_str % (np.mean(ranks), np.std(ranks)))

In [3]:
reload(rec)

eval_stat = dict()
eval_ranks = dict()

def register(params, typ, ranks):
    print(eval_str % (np.mean(ranks), np.std(ranks), params))
    key = tuple(params + [typ])
    eval_stat[key] = (np.mean(ranks), np.std(ranks))
    eval_ranks[key] = ranks
    
seed = random.randint(0,1000)
eval_str = 'mean relative rank %.4f, std %.3f for params %s'

seed = 15
samples = 2000

rmdr = rec.build_recommender_from_triples(
    user_item_rating_triples, min_item_freq=5, min_user_freq=2)

start = time.time()

ranks = rec.ranking_stat(rmdr, samples, seed, no_mat=True, no_bias=False)
register([None, None], 'nomat', ranks)

for alpha in 3 * 1.2 ** np.arange(-4, 5):
    ranks = rec.ranking_stat(rmdr, samples, seed, alpha=alpha, no_mat=False, no_bias=True)
    register(['%3.2f' % alpha, None], 'nobias', ranks)

    for beta in 0.05 * 2. ** np.arange(-4, 5):
        params = ['%3.1f' % alpha, '%4.3f' % beta]

        ranks = rec.ranking_stat(rmdr, samples, seed, alpha=alpha, beta=beta, no_mat=False, no_bias=False)
        register(['%3.2f' % alpha, '%4.3f' % beta], 'normal', ranks)
print('took {:.2f} seconds per parameter set'.format((time.time()-start) / len(eval_stat)))

original counts: 96735 users, 319762 items
construction took 1.05 secs
mean relative rank 0.2447, std 0.267 for params [None, None]
mean relative rank 0.1679, std 0.230 for params ['1.45', None]
mean relative rank 0.1638, std 0.223 for params ['1.45', '0.003']
mean relative rank 0.1638, std 0.223 for params ['1.45', '0.006']
mean relative rank 0.1638, std 0.223 for params ['1.45', '0.013']
mean relative rank 0.1637, std 0.223 for params ['1.45', '0.025']
mean relative rank 0.1637, std 0.223 for params ['1.45', '0.050']
mean relative rank 0.1638, std 0.223 for params ['1.45', '0.100']
mean relative rank 0.1638, std 0.223 for params ['1.45', '0.200']
mean relative rank 0.1637, std 0.223 for params ['1.45', '0.400']
mean relative rank 0.1636, std 0.223 for params ['1.45', '0.800']
mean relative rank 0.1661, std 0.230 for params ['1.74', None]
mean relative rank 0.1620, std 0.223 for params ['1.74', '0.003']
mean relative rank 0.1619, std 0.223 for params ['1.74', '0.006']
mean relative ra

In [ ]:
import pickle
with open('./eval-seed3-muf2-mif5-samples1k.pickle', 'wb') as f:
    pickle.dump((eval_stat, eval_ranks), f)
    

In [4]:
for r in sorted(eval_stat.items(), key=lambda x: x[1][0]):
    print(r[0], '%6.4f, std %4.2f' % r[1])

('2.50', '0.100', 'normal') 0.1600, std 0.22
('2.50', '0.050', 'normal') 0.1602, std 0.22
('2.08', '0.400', 'normal') 0.1603, std 0.22
('2.50', '0.200', 'normal') 0.1604, std 0.22
('2.08', '0.800', 'normal') 0.1606, std 0.22
('2.50', '0.025', 'normal') 0.1606, std 0.22
('2.08', '0.200', 'normal') 0.1606, std 0.22
('2.08', '0.100', 'normal') 0.1607, std 0.22
('3.00', '0.013', 'normal') 0.1608, std 0.22
('3.00', '0.025', 'normal') 0.1608, std 0.22
('2.50', '0.013', 'normal') 0.1609, std 0.22
('2.08', '0.050', 'normal') 0.1609, std 0.22
('2.08', '0.003', 'normal') 0.1610, std 0.22
('2.08', '0.006', 'normal') 0.1610, std 0.22
('3.00', '0.006', 'normal') 0.1610, std 0.22
('2.08', '0.025', 'normal') 0.1611, std 0.22
('2.50', '0.003', 'normal') 0.1611, std 0.22
('2.50', '0.006', 'normal') 0.1611, std 0.22
('2.08', '0.013', 'normal') 0.1611, std 0.22
('2.50', '0.400', 'normal') 0.1613, std 0.22
('3.00', '0.050', 'normal') 0.1613, std 0.22
('3.00', '0.003', 'normal') 0.1613, std 0.22
('1.74', '